<a href="https://colab.research.google.com/github/Kyssen/CSC148/blob/main/LearnAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **IMPORTS**

In [ ]:
import pandas as pd
import numpy as np
import random 
import cv2
import os
import glob
import matplotlib.pyplot as plt


In [ ]:
#Importing data set from Kaggle
os.environ['KAGGLE_USERNAME'] = 'yusufmoola211'
os.environ['KAGGLE_KEY'] = '00c7ae4c784f63a68821345cd24f9474'
! kaggle datasets download -d ashishjangra27/face-mask-12k-images-dataset


100% 330M/330M [00:17<00:00, 21.1MB/s]
100% 330M/330M [00:17<00:00, 19.6MB/s]


In [ ]:
#Unzipping file
! unzip face-mask-12k-images-dataset.zip

In [ ]:
#Directories for the three files: Test, Train and Validation
root_dir = '/content/Face Mask Dataset'
train_directory = os.path.join(root_dir, 'Train') 
val_directory = os.path.join(root_dir, 'Validation')
test_directory = os.path.join(root_dir, 'Test')

#Getting Mask and no mask folder within the train directory
train_with_mask = os.path.join(train_directory, 'WithMask')
train_without_mask = os.path.join(train_directory, 'WithoutMask')

#Getting Mask and no mask folder within the valid directory
val_with_mask = os.path.join(val_directory, 'WithMask')
val_without_mask = os.path.join(val_directory, 'WithoutMask')

#Getting Mask and no mask folder within the test directory

test_with_mask = os.path.join(test_directory, 'WithMask')
test_without_mask = os.path.join(test_directory, 'WithoutMask')


In [ ]:
# Number of images in the training dataset
print("-------------------------------")
print('Training dataset image count:')
print("-------------------------------")
print("Number of Mask labeled images -", len(os.listdir(train_with_mask)))
print("Number of NonMask labeled images -", len(os.listdir(train_without_mask)))

# Number of images in the valid dataset
print("\n-------------------------------")
print('Valid dataset image count:')
print("-------------------------------")
print("Number of Mask labeled images -", len(os.listdir(val_with_mask)))
print("Number of NonMask labeled images -", len(os.listdir(val_without_mask)))

# Number of images in the test dataset
print("\n-------------------------------")
print('Test dataset image count:')
print("-------------------------------")
print("Number of Mask labeled images -", len(os.listdir(test_with_mask)))
print("Number of NonMask labeled images -", len(os.listdir(test_without_mask)))




-------------------------------
Training dataset image count:
-------------------------------
Number of Mask labeled images - 5000
Number of NonMask labeled images - 5000

-------------------------------
Valid dataset image count:
-------------------------------
Number of Mask labeled images - 400
Number of NonMask labeled images - 400

-------------------------------
Test dataset image count:
-------------------------------
Number of Mask labeled images - 483
Number of NonMask labeled images - 509


In [ ]:
#Training set with and without mask count
num_withmask_tr = len(os.listdir(train_with_mask))
num_withoutmask_tr = len(os.listdir(train_without_mask))

#Valid set with and without mask count
num_withmask_val = len(os.listdir(os.path.join(val_directory, 'WithMask')))
num_withoutmask_val = len(os.listdir(os.path.join(val_directory, 'WithoutMask')))

#Test set with and without mask count
num_withmask_test = len(os.listdir(os.path.join(test_directory, 'WithMask')))
num_withoutmask_test = len(os.listdir(os.path.join(test_directory, 'WithoutMask')))

#Totals
total_train = num_withmask_tr + num_withoutmask_tr
total_val = num_withmask_val + num_withoutmask_val
total_test = num_withmask_test + num_withoutmask_test


print("----------------------------------------")
print("                 TOTALS                 ")
print("----------------------------------------")
print("Number of images from Training set:", total_train)
print("Number of images from Validation set:", total_val)
print("Number of images from Validation set:", total_test)



----------------------------------------
                 TOTALS                 
----------------------------------------
Number of images from Training set: 10000
Number of images from Validation set: 800
Number of images from Validation set: 992


# **DATA** **VISUALIZATION**

In [ ]:
#Main directory that includes Test, Train and Validation dataset
path = root_dir
#Lists the folders in the root directory
os.listdir(path)


['Validation', 'Train', 'Test']

In [ ]:
image_links = []
mask_or_not = []
category = []

#loop through folders in root directory
for folder in os.listdir(path):
    #Loop through labels (WithMask and WithoutMask) in each folder of Test, Train and Validation
    for label in os.listdir(path + "/" + folder):
        #Gets images in each folder
        for image in glob.glob(path+ "/" + folder + "/" + label + "/" + "*.png"):
            image_links.append(image)
            mask_or_not.append(label)
            category.append(folder)


#Making the pandas dataframe            
data = pd.DataFrame({'image_links':image_links,'label':mask_or_not,'category':category})
#Outputting the first 4 columns of the dataframe as a sample view
data.head()

,image_links,label,category
0,/content/Face Mask Dataset/Validation/WithMask...,WithMask,Validation
1,/content/Face Mask Dataset/Validation/WithMask...,WithMask,Validation
2,/content/Face Mask Dataset/Validation/WithMask...,WithMask,Validation
3,/content/Face Mask Dataset/Validation/WithMask...,WithMask,Validation
4,/content/Face Mask Dataset/Validation/WithMask...,WithMask,Validation


In [ ]:
#Seperarting the data frame into train, test and val through filter operations
train_data =data[data['category']=='Train'].sample(frac=1)
test_data = data[data['category']=='Test'].sample(frac=1)
validation = data[data['category']=='Validation'].sample(frac=1) 

#btw frac=1 just means getting 100% of the row data to return

#Variables for the image paths of each set (Train, test and valid)
train_path = train_data.image_links
test_path = test_data.image_links
valid_path = validation.image_links






In [ ]:
#Function to plot the images
def plot_images(data):
    for index in range(5):

        #Getting image from dataframe, opencv used to read image from path
        plt.imshow(cv2.imread(data.iloc[index,0]))

        #Adding title to the image for the label and which category it belongs to
        plt.title('{}:{}'.format(data.iloc[index,1],data.iloc[index,2]))
        plt.show()

In [ ]:
plot_images(validation)
plot_images(train_data)
plot_images(test_data)

# **LABELS AND PREPROCESSING**

In [ ]:
import tensorflow as tf
# Define the classes
classes = ['WithMask', 'WithoutMask']

# Define a function to load and preprocess the images
def load_image(path):
    # Load the image and resize it
    img = cv2.imread(path)
    img = cv2.resize(img, (224,224))
    
    # Preprocess the image
    img = img / 255.0
    img = np.array(img)
    
    return img

In [ ]:
# Load the training images
train_images = []
train_labels = []
for c in classes:
    class_path = os.path.join(train_directory, c)
    for image_path in os.listdir(class_path):
        img = load_image(os.path.join(class_path, image_path))
        train_images.append(img)
        train_labels.append(classes.index(c))

# Convert the lists to numpy arrays
train_images = np.array(train_images)
train_labels = np.array(train_labels)

# Load the validation images
val_images = []
val_labels = []
for c in classes:
    class_path = os.path.join(val_directory, c)
    for image_path in os.listdir(class_path):
        img = load_image(os.path.join(class_path, image_path))
        val_images.append(img)
        val_labels.append(classes.index(c))

# Convert the lists to numpy arrays
val_images = np.array(val_images)
val_labels = np.array(val_labels)

# Load the test images
test_images = []
test_labels = []
for c in classes:
    class_path = os.path.join(test_directory, c)
    for image_path in os.listdir(class_path):
        img = load_image(os.path.join(class_path, image_path))
        test_images.append(img)
        test_labels.append(classes.index(c))

# Convert the lists to numpy arrays
test_images = np.array(test_images)
test_labels = np.array(test_labels)

# Convert the labels to categorical format
train_labels = tf.keras.utils.to_categorical(train_labels, len(classes))
val_labels = tf.keras.utils.to_categorical(val_labels, len(classes))
test_labels = tf.keras.utils.to_categorical(test_labels, len(classes))
    

# **Face Detection from image**

In [ ]:
import cv2
from google.colab import drive
drive.mount('/content/drive')
from google.colab.patches import cv2_imshow

In [ ]:
base_dir = '/content/drive/My Drive/UofT AI project google collab/'

In [ ]:


# Read the input image
img = cv2.imread(base_dir + 'img/people.jpg')
  
# convert to grayscale of each frames
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# read the haarcascade to detect the faces in an image
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt.xml')

# detects faces in the input image
faces = face_cascade.detectMultiScale(gray, 1.1, 4)
print('Number of detected faces:', len(faces))

# loop over all detected faces
if len(faces) > 0:
   for i, (x,y,w,h) in enumerate(faces):
      
      # To draw a rectangle in a face
      cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
      face = img[y:y+h, x:x+w]
      cv2_imshow(face)
      cv2.imwrite(f'face{i}.jpg', face)
      print(f"face{i}.jpg is saved")

# display the image with detected faces
cv2_imshow(img)
cv2.waitKey(0)
cv2.destroyAllWindows()